# Aufgabe 1



# Datenvorbereitung mit Python

In diesem Notebook laden wir den Datensatz `Aufgabe-1.csv` ein, identifizieren typische Probleme (Trennzeichen, fehlende Werte, falsche Typen, Inkonsistenzen in Strings, Ausreißer) und bereinigen sie schrittweise. Am Ende speichern wir die bereinigte Datei ab.



Zusammenfassung der Arbeitsschritte

- Format­erkennung mit csv.Sniffer
- Einlesen mit korrekten Parametern und on_bad_lines='warn'

- Inspektion (Typen, fehlende Werte, Ausreißer)
- Bereinigung
  - String-Säuberung und Vereinheitlichung
  - Entfernen von Null-Werten und Duplikaten

- Speichern des sauberen Datensatzes

### Bibliotheken & Imports

In [32]:
!python -m pip install pandas

In [33]:
import pandas as pd # Datenanalyse
import numpy as np # numerische Operationen
import csv # CSV-Parser
import re # für Regex-Operationen


# Wir nutzen pandas für DataFrames, numpy für numerische Operationen
# und das csv-Modul, um das Dateiformat (Delimiter, Quotechar) automatisch zu erkennen.

## 2. Dateiformat automatisch erkennen

Statt blind auf Komma oder Semikolon zu setzen, liest `csv.Sniffer()` eine Probe und ermittelt das Trennzeichen (`delimiter`) und das Zeichen für String-Markierung (`quotechar`).


In [34]:
with open('Data/Aufgabe-1.csv', newline='', encoding='utf-8') as f:
    sample = f.read(2048)
    dialect = csv.Sniffer().sniff(sample) 
    delimiter = dialect.delimiter
    quotechar = dialect.quotechar

print(f"Erkannter delimiter: {delimiter!r}, quotechar: {quotechar!r}")

Erkannter delimiter: ',', quotechar: "'"


## 3. Datensatz einlesen

Nun laden wir den CSV mit den ermittelten Parametern.  
- `sep=delimiter`: korrektes Spaltentrennzeichen  
- `quotechar=quotechar`: z. B. Anführungszeichen um Strings  
- `encoding='utf-8'`: Annahme; ggf. anpassen, falls Fehler auftreten.


In [35]:
df = pd.read_csv(
    'Data/Aufgabe-1.csv',
    sep=delimiter,
    quotechar=quotechar,
    encoding='utf-8',
    on_bad_lines='warn'   # Zeilen mit falscher Anzahl Spalten nur warnen, statt Fehler zu werfen
)

# Erste Kontrolle
print("Shape:", df.shape)
df.head()


Shape: (18538, 89)


C:\Users\luisa\AppData\Local\Temp\ipykernel_24060\1823679723.py:1: ParserWarning: Skipping line 12922: expected 89 fields, saw 90

  df = pd.read_csv(
C:\Users\luisa\AppData\Local\Temp\ipykernel_24060\1823679723.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,Known As,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,...,LM Rating,CM Rating,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating
0,L. Messi,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,...,91,88,91,67,66,67,62,53,62,22
1,K. Benzema,Karim Benzema,91,91,64000000,"CF,ST",CF,France,https://cdn.sofifa.net/players/165/153/23_60.png,34,...,89,84,89,67,67,67,63,58,63,21
2,R. Lewandowski,Robert Lewandowski,91,91,84000000,ST,ST,Poland,https://cdn.sofifa.net/players/188/545/23_60.png,33,...,86,83,86,67,69,67,64,63,64,22
3,K. De Bruyne,Kevin De Bruyne,91,91,107500000,"CM,CAM",CM,Belgium,https://cdn.sofifa.net/players/192/985/23_60.png,31,...,91,91,91,82,82,82,78,72,78,24
4,K. Mbappé,Kylian Mbappé,91,95,190500000,"ST,LW",ST,France,https://cdn.sofifa.net/players/231/747/23_60.png,23,...,92,84,92,70,66,70,66,57,66,21


## 4. Erste Inspektion

- `df.info()`: Datentypen, Null-Werte  
- `df.describe()`: statistische Kennzahlen der numerischen Spalten  
- `df.isnull().sum()`: Anzahl fehlender Werte pro Spalte  

### Wofür?

- Fehlende Werte müssen wir später entweder entfernen oder sinnvoll ersetzen.
- Falsche Datentypen (z. B. Zahlen als Strings) müssen korrigiert werden.
- `on_bad_lines='warn'` fängt Zeilen mit unpassender Spaltenzahl und zeigt, wo Probleme sind.

In [36]:
# Datentypen und Null-Werte prüfen
df.info()

# Statistische Kennzahlen kurz anschauen
df.describe(include='all')

# Fehlende Werte zählen
print("Fehlende Werte pro Spalte:\n", df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18538 entries, 0 to 18537
Data columns (total 89 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Known As                     18538 non-null  object 
 1   Full Name                    18538 non-null  object 
 2   Overall                      18538 non-null  int64  
 3   Potential                    18538 non-null  int64  
 4   Value(in Euro)               18538 non-null  object 
 5   Positions Played             18538 non-null  object 
 6   Best Position                18538 non-null  object 
 7   Nationality                  18537 non-null  object 
 8   Image Link                   18538 non-null  object 
 9   Age                          18538 non-null  int64  
 10  Height(in cm)                18537 non-null  float64
 11  Weight(in kg)                18537 non-null  float64
 12  TotalStats                   18538 non-null  int64  
 13  BaseStats       

## 4b. Tiefergehende Typ-Analyse

Um zu erkennen, ob Spalten nicht den erwarteten Datentyp haben oder gemischte Typen enthalten, führen wir nun:

- Eine Übersicht über `df.dtypes`  

In [37]:
# Übersicht aller Spalten mit aktuellem dtype

dtype_df = pd.DataFrame({
    'Dtype aktuell': df.dtypes.astype(str)
})
print(dtype_df)


               Dtype aktuell
Known As              object
Full Name             object
Overall                int64
Potential              int64
Value(in Euro)        object
...                      ...
RWB Rating             int64
LB Rating              int64
CB Rating              int64
RB Rating              int64
GK Rating              int64

[89 rows x 1 columns]


In [38]:
# Für jede object-Spalte: welche Python-Typen kommen vor?
for col in df.select_dtypes(include=['object']).columns:
    type_counts = (
        df[col]
        .map(lambda x: type(x).__name__ if pd.notnull(x) else 'NaN')
        .value_counts()
    )
    
    print(f"\nSpalte '{col}' – Python-Typen:")
    print(type_counts)



Spalte 'Known As' – Python-Typen:
Known As
str    18538
Name: count, dtype: int64

Spalte 'Full Name' – Python-Typen:
Full Name
str    18538
Name: count, dtype: int64

Spalte 'Value(in Euro)' – Python-Typen:
Value(in Euro)
str    16384
int     2154
Name: count, dtype: int64

Spalte 'Positions Played' – Python-Typen:
Positions Played
str    18538
Name: count, dtype: int64

Spalte 'Best Position' – Python-Typen:
Best Position
str    18538
Name: count, dtype: int64

Spalte 'Nationality' – Python-Typen:
Nationality
str    18537
NaN        1
Name: count, dtype: int64

Spalte 'Image Link' – Python-Typen:
Image Link
str    18538
Name: count, dtype: int64

Spalte 'Club Name' – Python-Typen:
Club Name
str    18538
Name: count, dtype: int64

Spalte 'Club Position' – Python-Typen:
Club Position
str    18538
Name: count, dtype: int64

Spalte 'Contract Until' – Python-Typen:
Contract Until
str    18538
Name: count, dtype: int64

Spalte 'Club Jersey Number' – Python-Typen:
Club Jersey Number
str   

In [39]:
# Analyse der numerischen Spalten

# Wähle alle Spalten, die pandas als numerisch einstuft
num_cols = df.select_dtypes(include=['number']).columns.tolist()

for col in num_cols:
    # Mappe jeden Wert auf seinen Python-Typ, NaNs als 'NaN'
    type_counts = (
        df[col]
        .map(lambda x: type(x).__name__ if pd.notnull(x) else 'NaN')
        .value_counts()
    )
    print(f"\nSpalte '{col}' – Python-Typen:")
    print(type_counts)



Spalte 'Overall' – Python-Typen:
Overall
int    18538
Name: count, dtype: int64

Spalte 'Potential' – Python-Typen:
Potential
int    18538
Name: count, dtype: int64

Spalte 'Age' – Python-Typen:
Age
int    18538
Name: count, dtype: int64

Spalte 'Height(in cm)' – Python-Typen:
Height(in cm)
float    18537
NaN          1
Name: count, dtype: int64

Spalte 'Weight(in kg)' – Python-Typen:
Weight(in kg)
float    18537
NaN          1
Name: count, dtype: int64

Spalte 'TotalStats' – Python-Typen:
TotalStats
int    18538
Name: count, dtype: int64

Spalte 'BaseStats' – Python-Typen:
BaseStats
int    18538
Name: count, dtype: int64

Spalte 'Wage(in Euro)' – Python-Typen:
Wage(in Euro)
int    18538
Name: count, dtype: int64

Spalte 'Release Clause' – Python-Typen:
Release Clause
int    18538
Name: count, dtype: int64

Spalte 'Joined On' – Python-Typen:
Joined On
int    18538
Name: count, dtype: int64

Spalte 'Weak Foot Rating' – Python-Typen:
Weak Foot Rating
int    18538
Name: count, dtype: int

In [40]:
# Übersicht problematischer Spalten

# Liste, die Infos zu jeder Spalte sammelt
report = []

for col in df.columns:
    # Grundinfos
    missing = df[col].isnull().sum()
    dtype = df[col].dtype
    
    # Verteilung der tatsächlichen Python-Typen in der Spalte
    type_counts = (
        df[col]
        .map(lambda x: type(x).__name__ if pd.notnull(x) else 'NaN') 
        .value_counts()
    ) 
    # Heterogene Typen = mehr als ein Typ außer 'NaN'
    mixed = len(type_counts.drop(labels=['NaN'], errors='ignore')) > 1
    
    # Für object-Spalten: Anzahl der Einträge, die sich nicht in numerisch umwandeln lassen
    non_numeric = 0
    if dtype == 'object':
        coerced = pd.to_numeric(df[col], errors='coerce')
        non_numeric = ((coerced.isna()) & df[col].notna()).sum()
    
    report.append({
        'Spalte': col,
        'Dtype aktuell': str(dtype),
        'Missing values': missing,
        'Mixed types': mixed,
        'Non-numeric entries': non_numeric
    })

# DataFrame aus der Übersicht
report_df = pd.DataFrame(report)

# Nur Spalten mit mindestens einem Problem anzeigen
problems = report_df[
    (report_df['Missing values'] > 0) |
    (report_df['Mixed types']) 
]

# Ausgabe aller problematischen Spalten und ihrer Kennzahlen
print(problems.to_string(index=False))


                  Spalte Dtype aktuell  Missing values  Mixed types  Non-numeric entries
          Value(in Euro)        object               0         True                    3
             Nationality        object               1        False                18537
           Height(in cm)       float64               1        False                    0
           Weight(in kg)       float64               1        False                    0
International Reputation       float64               1        False                    0
             Positioning       float64               1        False                    0
     Goalkeeper Handling       float64               1        False                    0


In [41]:
#print(df["Nationality"].unique())
#print(df["Value(in Euro)"])

#print(df["Value(in Euro)"].map(lambda x: type(x).__name__ if pd.notnull(x) else 'NaN'), df["Value(in Euro)"])


## 5. Ausreißer und Null-Werte behandeln

- clean_value_in_euro-Funktion:
  - Abbruch bei Buchstaben sichert, dass fälschlich übergelaufener Text oder ein falsches Parsing nicht in unseren Zahlen landet.
  - Regex entfernt alle anderen unerwünschten Zeichen.
  - Apostroph wird als Tausender-Trenner gelöscht.
  - Es gibt keinen Standarddezimal Trenner mehr  
  - pd.to_numeric(..., errors='coerce') wandelt strings in Floats um, und setzt unlösbare Fälle in NaN.

- Damit ist die Spalte Value(in Euro) hinterher durchgängig ein int64 mit NaN für ungültige oder fehlende Einträge.


In [ ]:
# Rekonstruktion der "Value(in Euro)"-Spalte aus gesplitteten Zellen

# 1. Index der ursprünglichen "Value(in Euro)"-Spalte finden
cols       = df.columns.tolist()
val_idx    = cols.index('Value(in Euro)')
ncols      = len(cols) # Anzahl der Spalten

# 2. Für jede Zeile: Teile sammeln, bis eine Zelle mit Buchstaben kommt
reconstructed = []
# Zeilen durchlaufen
for i in range(len(df)): 
    parts = [] 
    j = val_idx # Start bei der "Value(in Euro)"-Spalte
    while j < ncols:
        cell = df.iat[i, j] # Zelle in der Zeile i, Spalte j
        # Stoppe, sobald eine Folgespalte Buchstaben, $ oder € enthält (neues Feld)
        if j > val_idx and pd.notnull(cell) and re.search(r'[A-Za-z$€]', str(cell)): 
            break
        parts.append(str(cell))
        j += 1
    reconstructed.append(''.join(parts))

# 3. In die DataFrame-Spalte zurückschreiben
df['Value(in Euro)'] = reconstructed


# 5. Anschließend mit der bekannte clean-Funktion in Integer wandeln
def clean_value_in_euro(raw):
    s = str(raw)
    # Entferne Fälle wie "00,€" oder "00.€"
    s = re.sub(r"([.,]00)€", "€", s)
    # entferne alles außer Ziffern, Komma, Punkt, Apostroph, Minus und €
    s = re.sub(r"[^0-9\.,'\-€]", "", s)
    # Entferne das Euro-Zeichen
    s = s.replace("€", "")
    # Apostroph löschen, Komma → Punkt
    s = s.replace("'", "").replace(",", "").replace(".", "")
    return pd.to_numeric(s, errors='coerce')

df['Value(in Euro)'] = df['Value(in Euro)'].apply(clean_value_in_euro)

# Kontrolle
print(df['Value(in Euro)'].head(10))


0     54000000
1     64000000
2     84000000
3    107500000
4    190500000
5    115500000
6     90000000
7     13500000
8     41000000
9     98000000
Name: Value(in Euro), dtype: int64


## Datentyp-Korrekturen und Bereinigung

- Missing-Value-Marker: Wir ersetzen alle gängigen Platzhalter ('', '-', 'NA' etc.) durch NA.
- Nullwerte und Negative Werte werden zu NA
- Duplikatentfernung 


In [43]:
# Platzhalter durch NA ersetzen – funktioniert für alle Spalten
df = df.replace(
    to_replace=[
        "N/A", "n/a", "NaN", "nan", "NULL", "null",
        "None", "none", "0"
    ],
    value='NA'
)

# Negative Werte und isolierte Minuszeichen in NA umwandeln
def replace_negatives_and_minus(value):
    if isinstance(value, (int, float)) and value < 0:
        return 'NA'
    if isinstance(value, str) and value.strip() == "-": 
        return 'NA'
    return value

df = df.map(replace_negatives_and_minus)

df = df.drop_duplicates()  # Duplikate entfernen

## 6. Bereinigten Datensatz speichern

Am Ende speichern wir die bereinigte CSV ab. So bleibt das Original unverändert und wir haben jederzeit den sauberen Datensatz griffbereit.


In [45]:

df.to_csv(
    'Aufgabe-1_clean.csv',
    index=False,
    sep=delimiter,
    encoding='utf-8'
)
print("Bereinigte Datei 'Aufgabe-1_clean.csv' erstellt.")

df.shape

Bereinigte Datei 'Aufgabe-1_clean.csv' erstellt.


(18419, 89)